In [ ]:
import json
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import re
from requests.compat import urlparse, urljoin
import sqlite3
from requests import Session
from datetime import datetime, timedelta 

In [ ]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params = params if method.upper() == 'GET' else '',
                               data = params if method.upper() == 'POST' else '',
                               headers = headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60) # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [ ]:
conn = sqlite3.connect('news2.db')
cur = conn.cursor()
cur.executescript('''
    DROP TABLE IF EXISTS head;
    CREATE TABLE head(
        pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        head TEXT NOT NULL,
        cdate TEXT NOT NULL,
        wdate TEXT NOT NULL,
        ref INTEGER NOT NULL,
        page INTEGER NOT NULL
    );
''')

cur.executescript('''
    DROP TABLE IF EXISTS history;
    CREATE TABLE history(
        pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        seen TEXT NOT NULL,
        ref INTEGER NOT NULL
    );
''')

In [ ]:
seen = list()
headers = { 
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
}
params = {
    '__VIEWSTATE': '/wEPDwULLTEyMTI4MTYxMjIPZBYCAgMPZBYCAgEPDxYCHgRUZXh0BQwyMDIwLiAwOC4gMThkZBgBBR5fX0NvbnRyb2xzUmVxdWlyZVBvc3RCYWNrS2V5X18WAQUGc2VhcmNomyRP70CWd1D8LnYgMGWG5YGgBBk=',
    '__VIEWSTATEGENERATOR': '8903B0C1',
    '__EVENTVALIDATION': '/wEWAwK95sj+CgK4y9eRBwLH0pL8CYEABvtNhMvEhlw5kV7Y98D0eTOW',
    'actionURL': 'http://www.dt.co.kr/eyescrap/eyescrapAction.html',
    # 'p_date': '2020-08-13'
}

# 시간변수 만들기
start_date = datetime.strptime('2019-08-17', '%Y-%m-%d') 
end_date = datetime.today()

str_date_list = [] 
while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'): 
    str_date_list.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)

url = 'http://papers.eyescrap.com/dt/list.aspx?actionURL=http%3a%2f%2fwww.dt.co.kr%2feyescrap%2feyescrapAction.html&p_date=' #주소 합치는거 고치기
urls = list()
for i in str_date_list:
    u = url + i
    urls.append(u)      

count = 0
    
while urls:
    try:
        count += 1
        seed = urls.pop(-1)

        resp = download(seed, params=params,headers=headers, method='GET')
        dom = BeautifulSoup(resp.text, 'html.parser')
        seen.append(seed)
        cur.execute('''
            INSERT INTO history(seen, ref) VALUES(?, 303)
        ''', [seed])
        
        
        if dom.select('#form1 > table > tr > td > table > tr > td > table > tr > td > table > tr > td > a')  != None:
            head = [_.text.strip() for _ in dom.select('#form1 > table > tr > td > table > tr > td > table > tr > td > table > tr > td > a') 
                     if _.text.strip()!='' and _.text.strip()!='[광고]']
            wdate = re.search('p_date=(\d+-\d+-\d+)', urlparse(seed).query).group(1)
            cdate = str(datetime.now()).split('.')[0]
            page = '1'
            for _ in range(0,len(head)):
                cur.execute('''
                    INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,303)
                    ''', [head[_], wdate, cdate, page])
                conn.commit()

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print('Error',e)